# Julia Serial Connection
## SerialPorts

In [117]:
using SerialPorts
list_serialports()

33-element Array{String,1}:
 "/dev/ttyACM23"
 "/dev/ttyS0"   
 "/dev/ttyS1"   
 "/dev/ttyS10"  
 "/dev/ttyS11"  
 "/dev/ttyS12"  
 "/dev/ttyS13"  
 "/dev/ttyS14"  
 "/dev/ttyS15"  
 "/dev/ttyS16"  
 "/dev/ttyS17"  
 "/dev/ttyS18"  
 "/dev/ttyS19"  
 ⋮              
 "/dev/ttyS27"  
 "/dev/ttyS28"  
 "/dev/ttyS29"  
 "/dev/ttyS3"   
 "/dev/ttyS30"  
 "/dev/ttyS31"  
 "/dev/ttyS4"   
 "/dev/ttyS5"   
 "/dev/ttyS6"   
 "/dev/ttyS7"   
 "/dev/ttyS8"   
 "/dev/ttyS9"   

## Open the serial connection

In [118]:
ser=SerialPort("/dev/ttyACM23", 9600)

SerialPort("/dev/ttyACM23", 9600, 8, "N", 1, nothing, false, false, false, PyObject Serial<id=0x7fe964484160, open=True>(port='/dev/ttyACM23', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False))

In [119]:
close(ser)

SerialPort("/dev/ttyACM23", 9600, 8, "N", 1, nothing, false, false, false, PyObject Serial<id=0x7fe964484160, open=False>(port='/dev/ttyACM23', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False))

To send a command

- clear the buffer
- write command with ser.write('c')
- wait for a response.

To read back the response for a short response, like the time taken message returned by 't'

- Use ser.read_all()

To read back the response when the micro needs to send lots of data

- Use ser.readline() or ser.read() in a loop. If you read fast enough, you can empty the buffer prematurely so
- check if the bufer is empty, and if it is, wait and check again before exiting the loop. If not
- Keep reading until the buffer stays empty

## Reading and sending 20k samples

In [11]:
a = "" # Keep bytes as a string
i = 0
b=""

# clear buffer
while bytesavailable(ser) > 0
    r = readavailable(ser)
end  

r = readavailable(ser) # clear serial buffer
# Get the values
write(ser,"b") # See code for command meanings
println("Command Sent")

while bytesavailable(ser) < 1
    continue # wait for a response
end  

while true
    if bytesavailable(ser) < 1
        sleep(0.005) # Wait and check again
        if bytesavailable(ser) < 1
            println("Finished Reading")
            break
        end
    end
    b = readavailable(ser) # Using read() is much slower due to extra delays
    a = string(a,b)
    i += 1    
end

println("Sample complete")
    
print(length(a)) # Should be 20000 readings, 2 bytes per reading (so 40k bytes)

Command Sent
Finished Reading
Sample complete
194

In [8]:
b

""

In [57]:
N = length(a)
x=zeros(Int8,N)
for n = 1:N
    x[n]=(Int8(a[n]))
end

Base.InvalidCharError{Char}: Base.InvalidCharError{Char}('\xaa')

In [48]:
write(ser,"b")

1

In [49]:
bytesavailable(ser)

200

In [304]:
bytesavailable(ser)

0

In [14]:
write(ser,"x")

1

In [66]:
bytesavailable(ser)

0

In [65]:
r = readavailable(ser)

""

## Echo Back

In [10]:
NStart=0
NEnd=100
#breaks at 128


for n = NStart:NEnd
    #a = string("",n)
    #println("writing ",UInt8(n))
    write(ser,UInt8(n))
    sleep(0.001)
end
sleep(0.1)

BytesWritten=NEnd-NStart+1

println("Bytes written ",BytesWritten)

BytesAvailable=bytesavailable(ser)
println("Bytes available:",BytesAvailable)
r = readavailable(ser) 
println("Recieved string ", r)

println("Length of string read ",length(r))

x=zeros(UInt8,BytesAvailable)

for n = 1:BytesAvailable
    print(n," ")
    x[n]=(UInt8(r[n]))
    println(x[n])
end

Bytes written 101
Bytes available:101
Recieved string  	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcd
Length of string read 101
1 0
2 1
3 2
4 3
5 4
6 5
7 6
8 7
9 8
10 9
11 10
12 11
13 12
14 13
15 14
16 15
17 16
18 17
19 18
20 19
21 20
22 21
23 22
24 23
25 24
26 25
27 26
28 27
29 28
30 29
31 30
32 31
33 32
34 33
35 34
36 35
37 36
38 37
39 38
40 39
41 40
42 41
43 42
44 43
45 44
46 45
47 46
48 47
49 48
50 49
51 50
52 51
53 52
54 53
55 54
56 55
57 56
58 57
59 58
60 59
61 60
62 61
63 62
64 63
65 64
66 65
67 66
68 67
69 68
70 69
71 70
72 71
73 72
74 73
75 74
76 75
77 76
78 77
79 78
80 79
81 80
82 81
83 82
84 83
85 84
86 85
87 86
88 87
89 88
90 89
91 90
92 91
93 92
94 93
95 94
96 95
97 96
98 97
99 98
100 99
101 100


In [45]:
x

255-element Array{UInt8,1}:
 0x00
 0x01
 0x02
 0x03
 0x04
 0x05
 0x06
 0x07
 0x08
 0x09
 0x0a
 0x0b
 0x0c
    ⋮
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00

In [78]:
r = readavailable(ser) 
N=100
for n = 0:N
    #a = string("",n)
    write(ser,UInt8(n))
end

In [79]:
bytesavailable(ser)

302

In [30]:
close(ser)

SerialPort("/dev/ttyACM4", 9600, 8, "N", 1, nothing, false, false, false, PyObject Serial<id=0x7fe9644849b0, open=False>(port='/dev/ttyACM4', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False))